In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import LeaveOneOut
import statsmodels.formula.api as smf
from sklearn.preprocessing import MinMaxScaler

In [39]:
# load data
data = pd.read_csv('merge_features_filtered.csv')
data.head()

,Trope,CharacterName,ActorName,ActorDOB,MovieName,Genres,Decade,ReleaseDate,BoxOffice,ActorID,...,actorID=341,actorID=342,actorID=343,Eye Distance,Eye Position,Nose Length,Nose Width,Eyebrow Length,Face Shape,Cheek Bones
0,evil_character,Aaron Hallam,Benicio del Toro,1967.0,The Hunted,"['Thriller', 'Crime Fiction', 'Action', 'Drama']",2000.0,2003.0,45496534.0,137.0,...,0.892761,0.820569,0.772236,0.267677,0.948980,0.373469,0.200337,0.340909,1.212245,0.183502
1,mean_officer,Abel Turner,Samuel L. Jackson,1948.0,Lakeview Terrace,"['Thriller', 'Crime Thriller', 'Drama', 'Psych...",2000.0,2008.0,44653637.0,273.0,...,1.058803,0.971612,1.004091,0.286885,0.949698,0.344064,0.260656,0.336885,1.227364,0.198361
2,charismatic_charmer,Abu,Sabu Dastagir,1924.0,The Thief of Bagdad,"['Adventure', 'Costume drama', 'Action/Adventu...",1940.0,1940.0,NaN,72.0,...,0.867241,0.812289,0.863133,0.261780,0.961039,0.396104,0.183246,0.348168,1.240260,0.170157
3,charismatic_charmer,Achilles,Brad Pitt,1963.0,Troy,"['Sword and Sandal', 'Adventure', 'Costume dra...",2000.0,2004.0,497409852.0,193.0,...,0.842054,0.782616,0.786956,0.264151,0.939914,0.347639,0.188679,0.328302,1.137339,0.141509
4,laidback_freebird,Agatha,Samantha Morton,1977.0,Minority Report,"['Crime Fiction', 'Thriller', 'Science Fiction...",2000.0,2002.0,358372926.0,178.0,...,0.872403,0.907091,0.919244,0.244406,0.931193,0.410550,0.175559,0.327022,1.332569,0.166093


In [40]:
data.columns = data.columns.str.replace(' ', '')
data

,Trope,CharacterName,ActorName,ActorDOB,MovieName,Genres,Decade,ReleaseDate,BoxOffice,ActorID,...,actorID=341,actorID=342,actorID=343,EyeDistance,EyePosition,NoseLength,NoseWidth,EyebrowLength,FaceShape,CheekBones
0,evil_character,Aaron Hallam,Benicio del Toro,1967.0,The Hunted,"['Thriller', 'Crime Fiction', 'Action', 'Drama']",2000.0,2003.0,45496534.0,137.0,...,0.892761,0.820569,0.772236,0.267677,0.948980,0.373469,0.200337,0.340909,1.212245,0.183502
1,mean_officer,Abel Turner,Samuel L. Jackson,1948.0,Lakeview Terrace,"['Thriller', 'Crime Thriller', 'Drama', 'Psych...",2000.0,2008.0,44653637.0,273.0,...,1.058803,0.971612,1.004091,0.286885,0.949698,0.344064,0.260656,0.336885,1.227364,0.198361
2,charismatic_charmer,Abu,Sabu Dastagir,1924.0,The Thief of Bagdad,"['Adventure', 'Costume drama', 'Action/Adventu...",1940.0,1940.0,NaN,72.0,...,0.867241,0.812289,0.863133,0.261780,0.961039,0.396104,0.183246,0.348168,1.240260,0.170157
3,charismatic_charmer,Achilles,Brad Pitt,1963.0,Troy,"['Sword and Sandal', 'Adventure', 'Costume dra...",2000.0,2004.0,497409852.0,193.0,...,0.842054,0.782616,0.786956,0.264151,0.939914,0.347639,0.188679,0.328302,1.137339,0.141509
4,laidback_freebird,Agatha,Samantha Morton,1977.0,Minority Report,"['Crime Fiction', 'Thriller', 'Science Fiction...",2000.0,2002.0,358372926.0,178.0,...,0.872403,0.907091,0.919244,0.244406,0.931193,0.410550,0.175559,0.327022,1.332569,0.166093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489,charismatic_charmer,Wolverine,Hugh Jackman,1968.0,X-Men,"['Thriller', 'Science Fiction', 'Superhero mov...",2000.0,2000.0,296339527.0,117.0,...,0.877475,0.714068,0.795558,0.275124,0.967505,0.408805,0.199341,0.351730,1.272537,0.181219
490,charismatic_charmer,Woody Deane,Kevin Zegers,1984.0,It's a Boy Girl Thing,"['Romantic comedy', 'Romance Film', 'Comedy', ...",2000.0,2006.0,7385434.0,290.0,...,0.926724,0.833277,0.849686,0.259322,0.971933,0.424116,0.194915,0.363559,1.226611,0.174576
491,old_wise_quirky,Yoda,Frank Oz,1944.0,Star Wars Episode V: The Empire Strikes Back,"['Science Fiction', 'Adventure', 'Space opera'...",1980.0,1980.0,538375067.0,311.0,...,0.960635,0.815970,0.802760,0.261250,0.955213,0.370044,0.218750,0.329375,1.174743,0.146875
492,skilled_badass,Zohan,Adam Sandler,1966.0,You Don't Mess with the Zohan,"['Action', 'Drama', 'Comedy']",2000.0,2008.0,201802891.0,57.0,...,0.926519,0.923931,0.892521,0.269966,0.948376,0.363109,0.233971,0.346457,1.031323,0.148481


In [41]:
data.loc[data.loc[data['ActorName'] == 'Sabu Dastagir'].index,'ActorHeight'] = 1.68
data.loc[data.loc[data['ActorName'] == 'Billie Whitelaw'].index,'ActorHeight'] = 1.57
data.loc[data.loc[data['ActorName'] == "Paige O'Hara"].index,'ActorHeight'] = 1.67
data.loc[data.loc[data['ActorName'] == "Alex Winter"].index,'ActorHeight'] = 1.70
data.loc[data.loc[data['ActorName'] == "Günter Lamprecht"].index,'ActorHeight'] = 1.82
data.loc[data.loc[data['ActorName'] == "Dan O'Herlihy"].index,'ActorHeight'] = 1.84
data.loc[data.loc[data['ActorName'] == "Brigitte Auber"].index,'ActorHeight'] = 1.61
data.loc[data.loc[data['ActorName'] == "Edmund Gwenn"].index,'ActorHeight'] = 1.65
data.loc[data.loc[data['ActorName'] == "Richard White"].index,'ActorHeight'] = 1.85
data.loc[data.loc[data['ActorName'] == "Peter Finch"].index,'ActorHeight'] = 1.75
data.loc[data.loc[data['ActorName'] == "David Murray"].index,'ActorHeight'] = 1.83
data.loc[data.loc[data['ActorName'] == "Lil Dagover"].index,'ActorHeight'] = 1.60
data.loc[data.loc[data['ActorName'] == "Lance Fenton"].index,'ActorHeight'] = 1.83
data.loc[data.loc[data['ActorName'] == "Jean Hagen"].index,'ActorHeight'] = 1.63
data.loc[data.loc[data['ActorName'] == "Wayne Morris"].index,'ActorHeight'] = 1.88
data.loc[data.loc[data['ActorName'] == "Ronald Lacey"].index,'ActorHeight'] = 1.73
data.loc[data.loc[data['ActorName'] == "Norman Lloyd"].index,'ActorHeight'] = 1.78
data.loc[data.loc[data['ActorName'] == "Junjie Qin"].index,'ActorHeight'] = 1.80
data.loc[data.loc[data['ActorName'] == "Michael McShane"].index,'ActorHeight'] = 1.83
data.loc[data.loc[data['ActorName'] == "Steve Guttenberg"].index,'ActorHeight'] = 1.83
data.loc[data.loc[data['ActorName'] == "Zack Ward"].index,'ActorHeight'] = 1.78
data.loc[data.loc[data['ActorName'] == "Takashi Matsuyama"].index,'ActorHeight'] = 1.73
data.loc[data.loc[data['ActorName'] == "John Larch"].index,'ActorHeight'] = 1.85
data.loc[data.loc[data['ActorName'] == "Robbie Gee"].index,'ActorHeight'] = 1.81
data.loc[data.loc[data['ActorName'] == "Kevin Gage"].index,'ActorHeight'] = 1.80
data.loc[data.loc[data['ActorName'] == "John Lynch"].index,'ActorHeight'] = 1.77
data.loc[data.loc[data['ActorName'] == "Nicholas Bell"].index,'ActorHeight'] = 1.84

In [42]:
data_copy = data.copy()
filtered_data_encodings = data_copy.filter(regex='^encod').copy()
# filtered_data.loc[:, 'Trope', 'CharacterName', 'ActorName', 'Genres', 'ReleaseDate', 'BoxOffice', 'ActorID', 'ActorGender', 'ActorHeight', 'ActorAge'] = data['Trope']
filtered_data_encodings[['Trope', 'CharacterName', 'ActorName', 'Genres', 'ReleaseDate', 'BoxOffice', 'ActorGender', 'ActorHeight', 'ActorAge']] = data[['Trope', 'CharacterName', 'ActorName', 'Genres', 'ReleaseDate', 'BoxOffice', 'ActorGender', 'ActorHeight', 'ActorAge']]

In [43]:
X_data_encodings = filtered_data_encodings.drop(['Trope', 'CharacterName', 'ActorName', 'Genres', 'ReleaseDate', 'BoxOffice'], axis=1)
y_data_encodings = filtered_data_encodings[['Trope']]
X_num_encodings = pd.get_dummies(X_data_encodings)
X_num_encodings.isna().sum().sum() 

0

In [44]:
filtered_data_ratios = data_copy[['Trope', 'CharacterName', 'ActorName', 'Genres', 'ReleaseDate', 'BoxOffice', 'ActorGender', 'ActorHeight', 'ActorAge', 'EyeDistance', 'EyePosition', 'NoseLength', 'NoseWidth', 'EyebrowLength', 'FaceShape', 'CheekBones']].copy()

In [54]:
X_data_ratios = filtered_data_ratios.drop(['Trope', 'CharacterName', 'ActorName', 'Genres', 'ReleaseDate', 'BoxOffice'], axis=1)
X_data_ratios_scaled = X_data_ratios.copy()
X_data_ratios_scaled[['ActorHeight', 'ActorAge', 'EyeDistance', 'EyePosition', 'NoseLength', 'NoseWidth', 'EyebrowLength', 'FaceShape', 'CheekBones']] = MinMaxScaler().fit_transform(X_data_ratios_scaled[['ActorHeight', 'ActorAge', 'EyeDistance', 'EyePosition', 'NoseLength', 'NoseWidth', 'EyebrowLength', 'FaceShape', 'CheekBones']])
y_data_ratios = filtered_data_ratios[['Trope']]
X_num_ratios = pd.get_dummies(X_data_ratios)
X_num_ratios.isna().sum().sum() 
X_num_ratios_scaled = X_num_ratios.copy()
X_num_ratios_scaled[['ActorHeight', 'ActorAge', 'EyeDistance', 'EyePosition', 'NoseLength', 'NoseWidth', 'EyebrowLength', 'FaceShape', 'CheekBones']] = MinMaxScaler().fit_transform(X_num_ratios_scaled[['ActorHeight', 'ActorAge', 'EyeDistance', 'EyePosition', 'NoseLength', 'NoseWidth', 'EyebrowLength', 'FaceShape', 'CheekBones']])


KNN on ratios

In [55]:
#KNN with tuning of K

k_range = list(range(1, 20))
KNN_Classifier = KNeighborsClassifier()


# Create a parameter grid: map the parameter names to the values that should be searched
param_grid = dict(n_neighbors=k_range)

# perform the search with CV
loo = LeaveOneOut()
grid = GridSearchCV(KNN_Classifier, param_grid, cv=loo, scoring='accuracy')
grid.fit(X_num_ratios_scaled, np.ravel(y_data_ratios))
results = grid.cv_results_

# Print the best score and the best K
print("Best score: %0.2f" % grid.best_score_)
best_K = grid.best_estimator_.get_params()['n_neighbors']
print("Best K:", best_K)

Best score: 0.22
Best K: 4


In [59]:
# Use the best parameters to fit the KNN model
KNN_Classifier = KNeighborsClassifier(n_neighbors=best_K)
KNN_Classifier.fit(X_num_ratios, np.ravel(y_data_ratios))

# Make predictions
y_pred = cross_val_predict(KNN_Classifier, X_num_ratios, np.ravel(y_data_ratios), cv=loo)

report = classification_report(y_data_ratios, y_pred)
print(report)

                     precision    recall  f1-score   support

charismatic_charmer       0.23      0.50      0.31        54
      crazy_fighter       0.09      0.17      0.11        36
  crazy_jealous_guy       0.06      0.08      0.07        25
    dumb_and_clumsy       0.28      0.31      0.29        36
   emotional_damage       0.07      0.06      0.06        17
     evil_character       0.00      0.00      0.00        26
               jock       0.00      0.00      0.00        27
  laidback_freebird       0.36      0.40      0.38        35
              loser       0.00      0.00      0.00        25
       mean_officer       0.03      0.02      0.03        48
    old_wise_quirky       0.33      0.17      0.22        24
   respected_leader       0.18      0.12      0.14        25
shallow_and_popular       0.48      0.39      0.43        31
           sidekick       0.00      0.00      0.00        21
     skilled_badass       0.23      0.14      0.18        50
        tech_genius    

tropes with f score higher than 20 : charismatic_charmer (CC), dumb_and_clumsy (DC), laidback_freebird (LF), old_wise_quirky (OWQ), shallow_and_popular (SP)

In [64]:
y_data_CC = y_data_ratios['Trope'].apply(lambda x: 1 if x == 'charismatic_charmer' else 0)
y_data_DC = y_data_ratios['Trope'].apply(lambda x: 1 if x == 'dumb_and_clumsy' else 0)
y_data_LF = y_data_ratios['Trope'].apply(lambda x: 1 if x == 'laidback_freebird' else 0)
y_data_OWQ = y_data_ratios['Trope'].apply(lambda x: 1 if x == 'old_wise_quirky' else 0)
y_data_SP = y_data_ratios['Trope'].apply(lambda x: 1 if x == 'shallow_and_popular' else 0)

In [60]:
# Get column names from X_num_ratios
column_names = '+'.join(X_num_ratios_scaled.columns)

inputs = 'y_data_CC ~ ' + column_names

# Perform OLS regression
lin_reg_is_CC = smf.ols(inputs, data=X_num_ratios_scaled).fit()
print(lin_reg_is_CC.summary())

print(inputs)

                            OLS Regression Results                            
Dep. Variable:              y_data_CC   R-squared:                       0.066
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     3.390
Date:                Wed, 20 Dec 2023   Prob (F-statistic):           0.000269
Time:                        17:06:25   Log-Likelihood:                -108.86
No. Observations:                 494   AIC:                             239.7
Df Residuals:                     483   BIC:                             285.9
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 0.13

In [61]:
# Get column names from X_num_ratios
column_names = '+'.join(X_num_ratios_scaled.columns)

inputs = 'y_data_DC ~ ' + column_names

# Perform OLS regression
lin_reg_is_CC = smf.ols(inputs, data=X_num_ratios_scaled).fit()
print(lin_reg_is_CC.summary())

print(inputs)

                            OLS Regression Results                            
Dep. Variable:              y_data_DC   R-squared:                       0.128
Model:                            OLS   Adj. R-squared:                  0.110
Method:                 Least Squares   F-statistic:                     7.072
Date:                Wed, 20 Dec 2023   Prob (F-statistic):           2.06e-10
Time:                        17:07:51   Log-Likelihood:                -1.6181
No. Observations:                 494   AIC:                             25.24
Df Residuals:                     483   BIC:                             71.46
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 0.15

In [65]:
# Get column names from X_num_ratios
column_names = '+'.join(X_num_ratios_scaled.columns)

inputs = 'y_data_LF ~ ' + column_names

# Perform OLS regression
lin_reg_is_CC = smf.ols(inputs, data=X_num_ratios_scaled).fit()
print(lin_reg_is_CC.summary())

print(inputs)

                            OLS Regression Results                            
Dep. Variable:              y_data_LF   R-squared:                       0.077
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     4.003
Date:                Wed, 20 Dec 2023   Prob (F-statistic):           2.79e-05
Time:                        17:09:22   Log-Likelihood:                -9.2846
No. Observations:                 494   AIC:                             40.57
Df Residuals:                     483   BIC:                             86.80
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 0.14

In [66]:
# Get column names from X_num_ratios
column_names = '+'.join(X_num_ratios_scaled.columns)

inputs = 'y_data_OWQ ~ ' + column_names

# Perform OLS regression
lin_reg_is_CC = smf.ols(inputs, data=X_num_ratios_scaled).fit()
print(lin_reg_is_CC.summary())

print(inputs)

                            OLS Regression Results                            
Dep. Variable:             y_data_OWQ   R-squared:                       0.073
Model:                            OLS   Adj. R-squared:                  0.053
Method:                 Least Squares   F-statistic:                     3.778
Date:                Wed, 20 Dec 2023   Prob (F-statistic):           6.45e-05
Time:                        17:10:56   Log-Likelihood:                 76.994
No. Observations:                 494   AIC:                            -132.0
Df Residuals:                     483   BIC:                            -85.76
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                -0.03

In [67]:
# Get column names from X_num_ratios
column_names = '+'.join(X_num_ratios_scaled.columns)

inputs = 'y_data_SP ~ ' + column_names

# Perform OLS regression
lin_reg_is_CC = smf.ols(inputs, data=X_num_ratios_scaled).fit()
print(lin_reg_is_CC.summary())

print(inputs)

                            OLS Regression Results                            
Dep. Variable:              y_data_SP   R-squared:                       0.311
Model:                            OLS   Adj. R-squared:                  0.297
Method:                 Least Squares   F-statistic:                     21.85
Date:                Wed, 20 Dec 2023   Prob (F-statistic):           1.04e-33
Time:                        17:12:43   Log-Likelihood:                 91.060
No. Observations:                 494   AIC:                            -160.1
Df Residuals:                     483   BIC:                            -113.9
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 0.14

KNN on encodings

#KNN with tuning of K

k_range = list(range(1, 20))

# Create a parameter grid: map the parameter names to the values that should be searched
param_grid = dict(n_neighbors=k_range)

# perform the search with CV
loo = LeaveOneOut()
grid = GridSearchCV(KNN_Classifier, param_grid, cv=loo, scoring='accuracy')
grid.fit(X_num_encodings, np.ravel(y_data_encodings))
results = grid.cv_results_

# Print the best score and the best K
print("Best score: %0.2f" % grid.best_score_)
best_K = grid.best_estimator_.get_params()['n_neighbors']
print("Best K:", best_K)